In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

开始使用Vertex AI TensorBoard#

在机器学习中，为了改进一些东西，通常需要能够对其进行测量。TensorBoard是一种工具，用于提供在机器学习工作流程中所需的测量和可视化。它使得能够追踪实验指标如损失和准确度，可视化模型图，并将嵌入投射到一个较低维度的空间，等等。

Vertex AI TensorBoard是开源TensorBoard项目的托管企业级版本。

这个快速入门将展示如何快速开始使用Vertex AI TensorBoard。

In [ ]:
# Installing the Vertex AI TensorBoard uploader

!python -m pip install google-cloud-aiplatform['tensorboard'] --upgrade

In [ ]:
import datetime

import tensorflow as tf

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
project_id = "YOUR PROJECT ID"
region = "YOUR REGION"

content_name = "vertex-ai-tb-get-started"

使用[MNIST](https://en.wikipedia.org/wiki/MNIST_database)数据集作为示例，对数据进行归一化，并编写一个函数来创建一个简单的Keras模型，用于将图像分类为10个类别。

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


def create_model():
    return tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

使用TensorBoard与Keras Model.fit()

在使用Keras的[Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit)进行训练时，添加`tf.keras.callbacks.TensorBoard`回调可以确保日志被创建和存储。此外，使用`histogram_freq=1`可以在每个epoch中启用直方图计算（默认情况下关闭）

将日志放在一个带有时间戳的子目录中，以便轻松选择不同的训练运行。

In [ ]:
model = create_model()
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    x=x_train,
    y=y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=project_id, location=region)

tensorboard = aiplatform.Tensorboard.create(display_name=content_name)

TENSORBOARD_RESOURCE_NAME = tensorboard.resource_name

In [ ]:
!tb-gcp-uploader --one_shot=True  --tensorboard_resource_name $TENSORBOARD_RESOURCE_NAME -logdir="logs/fit" --experiment_name="test-experiment1"

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> --> 

请将上述英文文本翻译为中文： 

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

对展示的仪表板（顶部导航栏中的选项卡）的简要概述：

* **Scalars** 仪表板显示损失和指标随每个时期的变化。您还可以使用它来跟踪训练速度、学习率和其他标量值。
* **Graphs** 仪表板帮助您可视化您的模型。在这种情况下，显示了层的Keras图，可以帮助您确保它构建正确。
* **Distributions** 和 **Histograms** 仪表板显示张量随时间的分布。这可以帮助您可视化权重和偏差，并验证它们是否以预期的方式变化。

当您记录其他类型的数据时，附加的TensorBoard插件会自动启用。例如，Keras TensorBoard回调允许您记录图像和嵌入。您可以通过点击右上角的“未激活”下拉菜单查看TensorBoard中还有哪些其他插件可用。

使用 Vertex AI TensorBoard 与其他方法

使用像[`tf.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape)这样的方法进行训练时，使用`tf.summary`来记录所需的信息。

使用与上面相同的数据集，但将其转换为`tf.data.Dataset`以利用批处理功能：

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

培训代码遵循[高级快速入门](https://www.tensorflow.org/tutorials/quickstart/advanced)教程，但展示如何将指标记录到TensorBoard。选择损失和优化器：

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

创建具有状态的指标，可以用来在训练过程中累积数值，并在任何时点进行记录。

In [ ]:
# Define our metrics
train_loss = tf.keras.metrics.Mean("train_loss", dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("train_accuracy")
test_loss = tf.keras.metrics.Mean("test_loss", dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("test_accuracy")

定义训练和测试功能：

In [ ]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)


def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

设置摘要写入程序将摘要写入不同的日志目录。

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = "logs/gradient_tape/" + current_time + "/train"
test_log_dir = "logs/gradient_tape/" + current_time + "/test"
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

开始训练。使用`tf.summary.scalar()`记录训练/测试期间的指标（损失和准确率），在摘要作者的范围内将摘要写入磁盘。您可以控制记录哪些指标以及如何频繁记录。其他`tf.summary`函数可用于记录其他类型的数据。

In [ ]:
model = create_model()  # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar("loss", train_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar("loss", test_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", test_accuracy.result(), step=epoch)

    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(
        template.format(
            epoch + 1,
            train_loss.result(),
            train_accuracy.result() * 100,
            test_loss.result(),
            test_accuracy.result() * 100,
        )
    )

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

再次上传TensorBoard日志，这次将其指向新的日志目录并上传到不同的实验（test-experiment2）。

我们也可以通过持续上传TensorBoard日志来监视训练的进展，省略"one_shot"标志。

In [ ]:
!tb-gcp-uploader --one_shot=True  --tensorboard_resource_name $TENSORBOARD_RESOURCE_NAME --logdir="logs/gradient_tape" --experiment_name="test-experiment2"

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> --> 

请将上述英文文本翻译成中文： 
<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> -->

就是这样！你现在已经看到了如何通过Keras回调和`tf.summary`来使用Vertex AI TensorBoard，适用于更多自定义情况。